In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

# Please replace the brackets below with the drive location of your folder which included subfolders for images
# Sample path: /content/drive/My Drive/ImageClassification
PATH = '/content/drive/My Drive/NebulasWithoutDarkReflection'

In [ ]:
def create_model(base_model, num_classes):
    # Grab the last layer and add a few extra layers to it
    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    # Dense layer 1
    x=tf.keras.layers.Dense(1024,activation='relu', kernel_initializer=tf.keras.initializers.VarianceScaling, use_bias=True)(x)

    # Final layer with softmax activation
    preds=tf.keras.layers.Dense(num_classes,activation='softmax', kernel_initializer=tf.keras.initializers.VarianceScaling, use_bias=False

)(x) 
    
    # Create the final model
    model=Model(inputs=base_model.input,outputs=preds)
    return model


In [ ]:
# Import packages needed to create a imaage classification model
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.applications.resnet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model

from tensorflow.keras.preprocessing import image_dataset_from_directory


BATCH_SIZE = 40
IMG_SIZE = (224, 224)

# Create the data generation pipeline for training and validation
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(PATH,
                                                target_size=IMG_SIZE,
                                                color_mode='rgb',
                                                batch_size=BATCH_SIZE,
                                                class_mode='categorical',
                                                shuffle=True,
                                                subset = 'training')
validation_generator = train_datagen.flow_from_directory(PATH,
                                                target_size=IMG_SIZE,
                                                color_mode='rgb',
                                                batch_size=BATCH_SIZE,
                                                class_mode='categorical',
                                                shuffle=False,
                                                subset = 'validation')

# Download the model
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
for layer in base_model.layers:
    layer.trainable=False
"""
# If we want to set the first 20 layers of the network to be non-trainable
for layer in base_model.layers[:150]:
    layer.trainable=False
for layer in base_model.layers[150:]:
    layer.trainable=True
"""

# Specify the number of classes
num_classes = 3

# Create the base model
model = create_model(base_model,num_classes)

print(len(base_model.layers))

base_learning_rate = 0.0001 #decrease for different results; use excel sheet to note down results from each change to learning rate and epochs

opt = keras.optimizers.Adam(learning_rate=base_learning_rate)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
history_fine = model.fit(train_generator,
                        steps_per_epoch=step_size_train,
                        epochs=45, #<-- increase for higher accuracy
                        validation_data = validation_generator)


Found 187 images belonging to 3 classes.
Found 45 images belonging to 3 classes.
9420800/9406464 [==============================] - 0s 0us/step
154
Epoch 1/45
4/4 [==============================] - 41s 7s/step - loss: 1.0801 - accuracy: 0.4354 - val_loss: 0.8760 - val_accuracy: 0.5333
Epoch 2/45
4/4 [==============================] - 6s 2s/step - loss: 0.8346 - accuracy: 0.7143 - val_loss: 0.7656 - val_accuracy: 0.6889
Epoch 3/45
4/4 [==============================] - 6s 2s/step - loss: 0.7357 - accuracy: 0.7551 - val_loss: 0.7158 - val_accuracy: 0.7333
Epoch 4/45
4/4 [==============================] - 6s 2s/step - loss: 0.6567 - accuracy: 0.7937 - val_loss: 0.6855 - val_accuracy: 0.6889
Epoch 5/45
4/4 [==============================] - 6s 2s/step - loss: 0.5997 - accuracy: 0.7687 - val_loss: 0.6869 - val_accuracy: 0.7556
Epoch 6/45
4/4 [==============================] - 6s 2s/step - loss: 0.5370 - accuracy: 0.8163 - val_loss: 0.6866 - val_accuracy: 0.7556
Epoch 7/45
4/4 [=============

In [ ]:

# Populating matrics -> accuracy & loss
acc = history_fine.history['accuracy']
val_acc = history_fine.history['val_accuracy']

loss = history_fine.history['loss']
val_loss = history_fine.history['val_loss']

print('Training Accuracy: ', acc)
print('Validation Accuracy: ', acc)
print('Training Loss: ', loss)
print('Validation Loss: ', val_loss)


Training Accuracy:  [0.26696833968162537, 0.4615384638309479, 0.5927602052688599, 0.6742081642150879, 0.692307710647583, 0.7692307829856873, 0.7873303294181824, 0.8280543088912964, 0.8280543088912964, 0.8461538553237915, 0.8597285151481628, 0.8733031749725342, 0.8823529481887817, 0.8959276080131531, 0.8979591727256775, 0.9004524946212769, 0.9230769276618958, 0.918552041053772, 0.9321267008781433, 0.9366515874862671, 0.9457013607025146, 0.959276020526886, 0.9714285731315613, 0.9683257937431335, 0.9673469662666321, 0.9773755669593811, 0.9773755669593811, 0.9864253401756287, 0.9864253401756287, 0.9954751133918762, 0.9819004535675049, 0.9819004535675049, 0.9954751133918762, 0.9909502267837524, 0.9954751133918762, 0.9954751133918762, 0.9909502267837524, 0.9954751133918762, 0.9954751133918762, 0.9954751133918762, 0.9954751133918762, 0.9954751133918762, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Validation Accuracy:  [0.26696833968162537, 0.4615384638309479, 0.5927602052688599, 0.674208164215087

In [ ]:

# Predicting code for an image
from tensorflow.keras.preprocessing import image
# Please replace the brackets below with the location of your image which need to predict
img_path = '/content/drive/My Drive/Nebulas/Dark/hubble386.jpeg'
img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)
img_preprocessed = preprocess_input(img_batch)
prediction = model.predict(img_preprocessed)
print(prediction)


[[9.7099048e-01 3.4337512e-03 7.7143624e-05 2.5227439e-02 2.7117057e-04]]
